In [1]:
import sys
from pathlib import Path

# change to your NFF path
sys.path.insert(0, "..")
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../")

import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler

from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate

import argparse
from sigopt import Connection

from train import train
from forceconv import *

from MD17data import *

from forcepai import ForcePai
# from nff.nn.models import Painn

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-logdir", type=str, default='./output')
parser.add_argument("-device", type=int, default=0)
parser.add_argument("-data", type=str, default='ethanol_dft')
params = vars(parser.parse_args([]))

DEVICE = params['device']
OUTDIR = '{}/{}/sandbox'.format(params['logdir'], 'test_ForcePai')

BATCH_SIZE = 10
lr = 1e-5
n_epochs = 100

In [3]:
data = get_MD17data(params['data'])
dataset = pack_MD17data(data, 10000)

In [4]:
train, val, test = split_train_validation_test(dataset, val_size=0.05, test_size=0.85)
train_loader = DataLoader(train, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, collate_fn=collate_dicts)

In [5]:
modelparams = {"feat_dim": 128,
              "activation": "swish",
              "n_rbf": 20,
              "cutoff": 5.0,
              "num_conv": 3,
              "output_keys": ["energy"],
              "grad_keys": ["energy_grad"],
               # whether to sum outputs from all blocks in the model
               # or just the final output block. False in the original
               # implementation
              "skip_connection": False,
               # Whether the k parameters in the Bessel basis functions
               # are learnable. False originally
              "learnable_k": False,
               # dropout rate in the convolution layers, originally 0
               "conv_dropout": 0.0,
               # dropout rate in the readout layers, originally 0
               "readout_dropout": 0.0,
               # dictionary of means to add to each output key
               # (this is optional - if you don't supply it then
               # nothing will be added)
               # "means": {"energy": train.props['energy'].mean().item()},
               # dictionary of standard devations with which to 
               # multiply each output key
               # (this is optional - if you don't supply it then
               # nothing will be multiplied)
               # "stddevs": {"energy": train.props['energy'].std().item()}
              }
model = ForcePai(modelparams)
# model = get_model(modelparams, model_type="Painn")

In [6]:
loss_fn = loss.build_mse_loss(loss_coef={'energy_grad': 1})
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = Adam(trainable_params, lr=lr)
train_metrics = [
        metrics.MeanAbsoluteError('energy_grad')
    ]

In [8]:
train_hooks = [
    hooks.MaxEpochHook(n_epochs),
    hooks.CSVHook(
        OUTDIR,
        metrics=train_metrics,
    ),
    hooks.PrintingHook(
        OUTDIR,
        metrics=train_metrics,
        separator = ' | ',
        time_strf='%M:%S'
    ),
    hooks.ReduceLROnPlateauHook(
        optimizer=optimizer,
        patience=20,
        factor=0.5,
        min_lr=1e-7,
        window_length=1,
        stop_after_min=True
    )
]

T = Trainer(
    model_path=OUTDIR,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
    checkpoint_interval=1,
    hooks=train_hooks,
    mini_batches=1
)

In [9]:
T.train(device=DEVICE, n_epochs=1000)

 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_energy_grad | GPU Memory (MB)


 99%|█████████▉| 99/100 [00:04<00:00, 20.23it/s]


12:04 |     1 |     1.000e-05 |   698.3097 |        634.2362 |         18.8060 |              46


 99%|█████████▉| 99/100 [00:04<00:00, 22.43it/s]


12:10 |     2 |     1.000e-05 |   653.9040 |        601.3732 |         18.2236 |              46


...

12:19 |     4 |     1.000e-05 |   534.6254 |        481.6557 |         15.7908 |              46


 99%|█████████▉| 99/100 [00:04<00:00, 23.88it/s]


12:24 |     5 |     1.000e-05 |   495.2409 |        451.6595 |         15.3353 |              46


 10%|█         | 10/100 [00:00<00:03, 23.54it/s]


KeyboardInterrupt: 

In [9]:
from nff.utils.cuda import batch_to, batch_detach
data = None
for batch in train_loader:
    data = batch_to(batch, DEVICE)
    break

In [10]:
rotate = torch.Tensor([[2**.5/2, -2**.5/2, 0],
                       [2**.5/2, 2**.5/2, 0],
                       [0, 0, 1]]).to(DEVICE)
rotate

tensor([[ 0.7071, -0.7071,  0.0000],
        [ 0.7071,  0.7071,  0.0000],
        [ 0.0000,  0.0000,  1.0000]], device='cuda:0')

In [12]:
data['nxyz'][:, 1:4] @= rotate

In [11]:
results = batch_to(batch_detach(model(data)), DEVICE)

In [13]:
new_results = batch_to(batch_detach(model(data)), DEVICE)

In [14]:
(new_results['energy_grad'] - (results['energy_grad'] @ rotate)).mean()

tensor(-7.7486e-08, device='cuda:0')

In [16]:
new_results['energy_grad']

tensor([[-3.0743e+00, -2.5356e+00,  6.8605e-01],
        [-3.7862e+00, -8.1742e+00, -7.3772e-01],
        [-9.2791e+00,  1.3064e+00,  9.5019e-01],
        [ 6.2518e+00,  1.5848e+01, -3.9878e+01],
        [ 7.4646e+00,  2.4472e+01,  8.0067e+00],
        [-7.5967e-01,  3.2987e+00,  2.2214e+00],
        [-2.7196e+00, -1.5982e+01,  2.6133e+01],
        [ 1.9472e+01, -1.5775e+01,  2.8369e+00],
        [-1.3569e+01, -2.4580e+00, -2.1894e-01],
        [-3.9159e+00, -8.4086e+00, -1.5095e+00],
        [-7.2181e+00,  2.1647e+00, -6.4895e+00],
        [ 2.4396e+00,  1.5267e+00, -2.2225e+00],
        [-2.4761e+00, -1.5227e+00,  2.0883e+00],
        [-8.3257e+00, -1.8099e+00,  6.4227e-01],
        [ 1.3366e+01,  1.9605e+01, -8.2195e+00],
        [ 1.6819e+01, -2.5273e+01,  1.3427e+01],
        [-9.3391e+00,  8.5677e+00,  1.2269e+01],
        [-1.3495e+00,  5.1496e+00, -9.9857e+00],
        [ 1.2127e+01,  1.2243e+01, -1.4270e+00],
        [ 8.6222e+00,  2.5295e+01, -1.7501e+00],
        [ 8.1527e+00

In [15]:
results['energy_grad'] @ rotate

tensor([[-3.0743e+00, -2.5356e+00,  6.8606e-01],
        [-3.7861e+00, -8.1742e+00, -7.3772e-01],
        [-9.2791e+00,  1.3064e+00,  9.5019e-01],
        [ 6.2518e+00,  1.5848e+01, -3.9878e+01],
        [ 7.4646e+00,  2.4472e+01,  8.0067e+00],
        [-7.5966e-01,  3.2987e+00,  2.2214e+00],
        [-2.7196e+00, -1.5982e+01,  2.6133e+01],
        [ 1.9472e+01, -1.5775e+01,  2.8369e+00],
        [-1.3569e+01, -2.4580e+00, -2.1894e-01],
        [-3.9159e+00, -8.4086e+00, -1.5095e+00],
        [-7.2181e+00,  2.1647e+00, -6.4895e+00],
        [ 2.4396e+00,  1.5267e+00, -2.2225e+00],
        [-2.4761e+00, -1.5227e+00,  2.0883e+00],
        [-8.3257e+00, -1.8100e+00,  6.4228e-01],
        [ 1.3366e+01,  1.9605e+01, -8.2195e+00],
        [ 1.6819e+01, -2.5273e+01,  1.3427e+01],
        [-9.3391e+00,  8.5678e+00,  1.2269e+01],
        [-1.3495e+00,  5.1496e+00, -9.9857e+00],
        [ 1.2127e+01,  1.2243e+01, -1.4270e+00],
        [ 8.6222e+00,  2.5295e+01, -1.7501e+00],
        [ 8.1527e+00

In [19]:
model.output_keys

['energy']